In [1]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="MNIST")
sc.version

'2.2.0.2.6.3.0-235'

In [2]:
import os
import glob
import numpy as np

#dataFolder='/user/gratienj/Data/MNIST'
dataFolder='/user/gratienj'
filename=os.path.join(dataFolder,'train.csv')

In [3]:
# Loads data. Be careful of indentations and whitespace
from pyspark.sql import SQLContext
import pyspark
from pyspark.sql import Row
from pyspark.sql.types import *
import csv

sql_c = SQLContext(sc)
#rdd = sc.textFile('/user/gratienj/exemple.csv').map(lambda line: line.split(","))
#fields = [StructField("age",StringType(),True),StructField("name", StringType(), True)]
#fields = [StructField("age",IntegerType(),True),StructField("name", StringType(), True)]
fields = [StructField("label", StringType(), True)]
for i in range(784):
    fields.append(StructField(str(i),IntegerType(),True))
schema=StructType(fields)
#df = sql_c.createDataFrame(rdd, schema)
df = sql_c.read.csv(filename,schema)
#df2 = rdd.toDF(schema).withColumn("agei", col("age").cast("integer")).drop("age")
#df = sql_c.read.csv('/home/gratienj/Data/MNIST/train.csv',header='false')
#df.show()
#df=df.withColumn("label", df["label"].cast(IntegerType()))
df=df.withColumn("label", df["label"].cast(IntegerType()))
df.printSchema()

root
 |-- label: integer (nullable = true)
 |-- 0: integer (nullable = true)
 |-- 1: integer (nullable = true)
 |-- 2: integer (nullable = true)
 |-- 3: integer (nullable = true)
 |-- 4: integer (nullable = true)
 |-- 5: integer (nullable = true)
 |-- 6: integer (nullable = true)
 |-- 7: integer (nullable = true)
 |-- 8: integer (nullable = true)
 |-- 9: integer (nullable = true)
 |-- 10: integer (nullable = true)
 |-- 11: integer (nullable = true)
 |-- 12: integer (nullable = true)
 |-- 13: integer (nullable = true)
 |-- 14: integer (nullable = true)
 |-- 15: integer (nullable = true)
 |-- 16: integer (nullable = true)
 |-- 17: integer (nullable = true)
 |-- 18: integer (nullable = true)
 |-- 19: integer (nullable = true)
 |-- 20: integer (nullable = true)
 |-- 21: integer (nullable = true)
 |-- 22: integer (nullable = true)
 |-- 23: integer (nullable = true)
 |-- 24: integer (nullable = true)
 |-- 25: integer (nullable = true)
 |-- 26: integer (nullable = true)
 |-- 27: integer (null

In [4]:
print('NB ROWS:',df.count())
print('NB COLUMNS',len(df.columns))

NB ROWS: 60000
NB COLUMNS 785


In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier as RF
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import functools
from pyspark.ml.feature import OneHotEncoder


In [6]:
cols = [ str(i) for i in range(784)]
assembler_features = VectorAssembler(inputCols=cols, outputCol='features')
labelIndexer = StringIndexer(inputCol='label', outputCol="label2")
tmp = [assembler_features, labelIndexer]
pipeline = Pipeline(stages=tmp)

In [7]:
allData = pipeline.fit(df).transform(df)
allData.cache()
trainingData, testData = allData.randomSplit([0.1,0.9], seed=0) # need to ensure same split for each time
#print("Distribution of Pos and Neg in trainingData is: ", trainingData.groupBy("label").count().take(3))

In [8]:
test2Data, test3Data = testData.randomSplit([0.1,0.9], seed=0)

In [ ]:
rf = RF(labelCol='label', featuresCol='features')
fit = rf.fit(trainingData)
transformed = fit.transform(test2Data)

In [ ]:
# Train a GBT model.
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

# Chain indexers and GBT in a Pipeline
#pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData)

model = gbt.fit(trainingData)

In [9]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [784, 256,128, 64, 10]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(trainingData)

# compute accuracy on the test set
result = model.transform(test2Data)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Py4JJavaError: An error occurred while calling o85.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 4.0 failed 1 times, most recent failure: Lost task 6.0 in stage 4.0 (TID 88, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1517)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1505)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1504)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1504)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
	at org.apache.spa

In [ ]:



# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only
